In [3]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

/home/garvit/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
from keras import backend as K
K.set_image_dim_ordering('th')

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [6]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam

In [7]:
PATH = os.getcwd()
data_path = PATH + '/data'
data_dir_list = os.listdir(data_path)

In [8]:
img_row = 128
img_col = 128
num_channel = 1
num_epoch = 20

In [9]:
num_classes = len(data_dir_list)

In [10]:
img_data_list=[]

In [11]:
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    
    # print img_list
    print 'Loaded the images of dataset-'+'{}\n'.format(dataset)
    
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)

Loaded the images of dataset-Diseased

Loaded the images of dataset-Normal



In [12]:
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

print img_data.shape

(56, 128, 128)


In [13]:
if num_channel == 1:
    if K.image_dim_ordering() == 'th':
        img_data = np.expand_dims(img_data, axis = 1)
        print img_data.shape
    else:
        img_data = np.expand_dims(img_data, axis = 4)
        print img_data.shape
# else:
#     if K.image_dim_ordering()=='th':
#         img_data=np.rollaxis(img_data,3,1)
#         print img_data.shape

(56, 1, 128, 128)


In [14]:
num_of_samples = img_data.shape[0]

In [15]:
labels = np.ones((num_of_samples,), dtype = 'int64')

In [16]:
labels[0:28] = 0
labels[28:] = 1

In [17]:
names = data_dir_list

In [18]:
Y = np_utils.to_categorical(labels, num_classes)

In [19]:
#Shuffle the dataset
x, y = shuffle(img_data, Y, random_state=2)

In [20]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 2)

In [21]:
input_shape=img_data[0].shape

model = Sequential()

model.add(Convolution2D(32, 3,3,border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

/home/garvit/anaconda3/envs/python2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/garvit/anaconda3/envs/python2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/garvit/anaconda3/envs/python2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [22]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])

In [23]:
# Training
hist = model.fit(X_train, y_train, batch_size = 16, nb_epoch = num_epoch, verbose = 1, validation_data = (X_test, y_test))

Train on 44 samples, validate on 12 samples
Epoch 1/20
44/44 [==============================] - 0s - loss: 3.3154 - acc: 0.5000 - val_loss: 0.7180 - val_acc: 0.5000
Epoch 2/20
44/44 [==============================] - 0s - loss: 0.7825 - acc: 0.5682 - val_loss: 0.6903 - val_acc: 0.5833
Epoch 3/20
44/44 [==============================] - 0s - loss: 0.6827 - acc: 0.5000 - val_loss: 0.6831 - val_acc: 0.5000
Epoch 4/20
44/44 [==============================] - 0s - loss: 0.6334 - acc: 0.6818 - val_loss: 0.6715 - val_acc: 0.5000

In [24]:
from keras.models import model_from_json
from keras.models import load_model

In [25]:
# Serialize model to JSON
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [26]:
# Serialize weights to HDFS
model.save_weights("model.h5")
print "Saved model to disk"

Saved model to disk
